In [16]:
# Class Graph

class Node:
  def __init__(self,name,militarybase,neighbours,population,weapons,isAttacked,aliens,pathcost,heuristiccost,totalcost,parent):
    self.name=name
    self.militarybase=militarybase
    self.weapons=weapons
    self.civilians=population
    self.neighbours=neighbours
    self.isAttacked=isAttacked
    self.aliens=aliens
    self.pathcost=pathcost
    self.heuristiccost=heuristiccost
    self.totalcost=totalcost
    self.parent=parent


In [17]:
# Main
map={'Alexandria': Node('City Alexandria', 'Fort ALexandria', [('City J', 101), ('City L', 85),('City K', 90)], 500,180,False,0,0,0,0,None),
     'City A': Node('City A', None, [('City B', 100), ('City H', 151)], 45,20,False,0,0,0,0,None),
     'City B': Node('City B', None, [('City A', 100), ('City H', 140),('City C',118)], 70,40,False,0,0,0,0,None),
     'City C': Node('City C', 'Fort C', [('City B', 118),('City D',111)], 100,70,False,0,0,0,0,None),
     'City D': Node('City D', None, [('City C', 111),('City E', 140)], 80,45,False,0,0,0,0,None),
     'City E': Node('City E', 'Fort E', [('City D', 140), ('City F', 120)], 130,90,False,0,0,0,0,None),
     'City F': Node('City F', None, [('City E', 120), ('City G', 146), ('City J',138)], 50,40,False,0,0,0,0,None),
     'City G': Node('City G', None, [('City F', 146),('City H',80),('City J',97)], 60,30,False,0,0,0,0,None),
     'City H': Node('City H', 'Fort H', [('City A',151),('City B', 140),('City G',80),('City I',99)], 115,85,False,0,0,0,0,None),
     'City I': Node('City I', None, [('Alexandria',210),('City H', 99)], 60,35,False,0,0,0,0,None),
     'City J': Node('City J', 'Fort J', [('Alexandria',101),('City F', 138),('City G',97)], 125,95,False,0,0,0,0,None),
     'City K': Node('City K', None, [('Alexandria', 90)],55,30,False,0,0,0,0,None),
     'City L': Node('City L', None, [('Alexandria', 85),('City M',140),('City P',98)], 65,35,False,0,0,0,0,None),
     'City M': Node('City M', 'Fort M', [('City L', 146),('City N',92)], 150,100,False,0,0,0,0,None),
     'City N': Node('City N', None, [('City M',92),('City O',87)], 80,45,False,0,0,0,0,None),
     'City O': Node('City O', None, [('City N', 87)], 45,25,False,0,0,0,0,None),
     'City P': Node('City P', 'Fort P', [('City L',98),('City Q',98)], 160,105,False,0,0,0,0,None),
     'City Q': Node('City Q', None, [('City P', 98),('City R',86)], 20,5,False,0,0,0,0,None),
     'City R': Node('City R', None, [('City Q', 86)], 45,15,False,0,0,0,0,None)}

In [18]:
# # Aliens Spawn Using BFS/DFS
from queue import Queue
import random

def printAliens():
    for name,node in map.items():
      print(f"City Name: {name}")
      print(f"Aliens: {node.aliens}")
      print()

def initialize_aliens(start, destination, population):
    explored = set()
    queue = Queue()
    queue.put((start, destination, population))

    while queue:
        curr, dest, pop = queue.get()
        aliens = random.randint(1, 100)
        curr.aliens=aliens
        explored.add((curr.name, aliens))

        if curr == dest:
            break

        for neighbour, distance in curr.neighbours:
            if neighbour not in explored:
                new_population = random.randint(1, 250)
                queue.put((map[neighbour], dest, new_population))


In [19]:
# A* Algorithm For Defense
from queue import PriorityQueue

def HeuristicFunction(node, destination,weapons):
    return (node.isAttacked * 10) + (node.aliens * 5) + (node.civilians * 0.5) + (node.weapons/weapons)

def ReturnPathCost(curr,next):
   return curr.totalcost+next.pathcost


def AStar(start, dest, map):
    frontier = PriorityQueue()
    visited = set()

    startNode = map[start]
    startNode.heuristiccost = HeuristicFunction(startNode, dest,startNode.weapons)
    startNode.totalcost = startNode.heuristiccost
    frontier.put((startNode.totalcost, startNode))

    while not frontier.empty():
        currcost, curr = frontier.get()

        if curr.name == dest:
            path = []
            while curr:
                path.append(curr.name)
                curr = curr.parent
            return path[::-1]

        if curr.name not in visited:
            visited.add(curr.name)

            for neighbour, edge_cost in curr.neighbours:
                neighbourNode = map[neighbour]

                PathCost = ReturnPathCost(curr,neighbourNode)
                HeuristicCost = HeuristicFunction(neighbourNode, dest,startNode.weapons)
                TotalCost = PathCost + HeuristicCost

                if neighbourNode.name not in visited:
                    frontier.put((TotalCost, neighbourNode))

                    if TotalCost < neighbourNode.totalcost or neighbourNode.totalcost == 0:
                        neighbourNode.totalcost = TotalCost
                        neighbourNode.parent = curr

    return None


In [20]:
# MAIN
import time

# Initialize Aliens Function
initialize_aliens(map['City D'],map['Alexandria'],120)
initialize_aliens(map['City O'],map['Alexandria'],60)
initialize_aliens(map['City R'],map['Alexandria'],40)

printAliens()

# Print Cities And Its details
for name, node in map.items():
    print(f"{name}:")
    print(f"  Military Base: {node.militarybase}")
    print(f"  Weapons: {node.weapons}")
    print(f"  Population: {node.civilians}")
    print(f"  Aliens: {node.aliens}")
    print(f"  Is City Attacked: {node.isAttacked}")
    print("  Neighbours:")
    for neighbour, distance in node.neighbours:
        print(f"    {neighbour}: {distance}Km")
    print()

# Simulate Invasion

for node_name, node in map.items():
    if node.aliens > 50:
        node.isAttacked = True
        print(f"{node.name} Under Attack!")
        temp = node.civilians
        count = min(temp * 2, node.aliens)
        node.aliens -= count
        temp-=45
        print(f"  {count} Aliens killed during the attack.")
        if temp>0:
            print(f" Civilian Casualties:{temp}")
        else:
            print("No Civilians Left")
        time.sleep(1)

print("Army Defense Strategic Movement")

# AStar to Find Best Path
start = 'City E'
dest= 'Alexandria'
path = AStar(dest,start,map)
path=path[::-1]
print(path)




City Name: Alexandria
Aliens: 6

City Name: City A
Aliens: 51

City Name: City B
Aliens: 74

City Name: City C
Aliens: 99

City Name: City D
Aliens: 33

City Name: City E
Aliens: 14

City Name: City F
Aliens: 70

City Name: City G
Aliens: 37

City Name: City H
Aliens: 53

City Name: City I
Aliens: 28

City Name: City J
Aliens: 30

City Name: City K
Aliens: 0

City Name: City L
Aliens: 78

City Name: City M
Aliens: 16

City Name: City N
Aliens: 3

City Name: City O
Aliens: 94

City Name: City P
Aliens: 49

City Name: City Q
Aliens: 8

City Name: City R
Aliens: 72

Alexandria:
  Military Base: Fort ALexandria
  Weapons: 180
  Population: 500
  Aliens: 6
  Is City Attacked: False
  Neighbours:
    City J: 101Km
    City L: 85Km
    City K: 90Km

City A:
  Military Base: None
  Weapons: 20
  Population: 45
  Aliens: 51
  Is City Attacked: False
  Neighbours:
    City B: 100Km
    City H: 151Km

City B:
  Military Base: None
  Weapons: 40
  Population: 70
  Aliens: 74
  Is City Attacked: Fa